In [ ]:
!pip install -U openai-whisper
!sudo apt update && sudo apt install ffmpeg -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.0 MB/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import subprocess
import os
from datetime import datetime

# Configurações
input_file = "videoplayback.mp4"
output_dir = "output_whisper"
model_name = "medium"  # Mude para "large-v3" se precisar de mais precisão
segment_duration = 3600  # 1 hora em segundos

# Verifica se o arquivo existe
if not os.path.exists(input_file):
    print(f"Erro: Arquivo '{input_file}' não encontrado!")
    exit()

# Cria diretório de saída
os.makedirs(output_dir, exist_ok=True)

# 1. Converter para WAV e dividir em partes de 1 hora
print("\nConvertendo e dividindo o áudio...")
try:
    # Primeiro converte para WAV
    temp_wav = os.path.join(output_dir, "temp_audio.wav")
    subprocess.run([
        "ffmpeg",
        "-i", input_file,
        "-vn", "-acodec", "pcm_s16le",
        "-ar", "16000", "-ac", "1",
        temp_wav
    ], check=True)

    # Depois divide em partes
    subprocess.run([
        "ffmpeg",
        "-i", temp_wav,
        "-f", "segment",
        "-segment_time", str(segment_duration),
        "-c", "copy",
        os.path.join(output_dir, "parte_%03d.wav")
    ], check=True)

    # Remove o arquivo temporário
    os.remove(temp_wav)

    print("Áudio dividido com sucesso em partes de 1 hora!")
except subprocess.CalledProcessError as e:
    print(f"Erro ao processar áudio: {e}")
    exit()

# 2. Transcrever cada parte
partes = sorted([f for f in os.listdir(output_dir) if f.startswith("parte_") and f.endswith(".wav")])
transcricao_completa = ""

for i, parte in enumerate(partes, 1):
    parte_path = os.path.join(output_dir, parte)
    output_name = os.path.splitext(parte)[0]  # Remove a extensão .wav
    output_txt = os.path.join(output_dir, f"{output_name}.txt")  # Usa o mesmo nome da parte

    print(f"\nTranscrevendo parte {i}/{len(partes)}: {parte}")
    start_time = datetime.now()

    try:
        # Executa o Whisper com output_file específico
        result = subprocess.run([
            "whisper",
            parte_path,
            "--language", "pt",
            "--model", model_name,
            "--output_dir", output_dir,
            "--output_format", "txt",
            "--task", "transcribe",
            "--verbose", "True"
        ], capture_output=True, text=True)

        # Verifica se o arquivo foi criado
        if not os.path.exists(output_txt):
            print(f"Aviso: O arquivo {output_txt} não foi criado!")
            print("Saída do Whisper:")
            print(result.stdout)
            print("Erros:")
            print(result.stderr)
            continue

        # Adiciona à transcrição completa
        with open(output_txt, 'r', encoding='utf-8') as f:
            conteudo = f.read()
            transcricao_completa += f"=== Parte {i} ===\n{conteudo}\n\n"

        print(f"Parte {i} concluída em {datetime.now() - start_time}")

    except Exception as e:
        print(f"Erro ao transcrever {parte}: {str(e)}")
        continue

# 3. Salvar transcrição completa
if transcricao_completa:
    final_output = os.path.join(output_dir, "transcricao_completa.txt")
    with open(final_output, 'w', encoding='utf-8') as f:
        f.write(transcricao_completa)

    print("\n" + "="*50)
    print(f"Transcrição completa salva em: {final_output}")
    print(f"Tamanho total do texto: {len(transcricao_completa.split())} palavras")
    print("Prévia da transcrição:")
    print(transcricao_completa[:500] + "...")

    # Faz download no Colab
    if 'google.colab' in str(get_ipython()):
        from google.colab import files
        files.download(final_output)
else:
    print("\nErro: Nenhuma transcrição foi gerada.")


Convertendo e dividindo o áudio...
Áudio dividido com sucesso em partes de 1 hora!

Transcrevendo parte 1/5: parte_000.wav
Parte 1 concluída em 0:09:08.647923

Transcrevendo parte 2/5: parte_001.wav
Parte 2 concluída em 0:15:12.597337

Transcrevendo parte 3/5: parte_002.wav
Parte 3 concluída em 0:13:46.040536

Transcrevendo parte 4/5: parte_003.wav
Parte 4 concluída em 0:12:34.820343

Transcrevendo parte 5/5: parte_004.wav
Parte 5 concluída em 0:05:18.048747

Transcrição completa salva em: output_whisper/transcricao_completa.txt
Tamanho total do texto: 32193 palavras
Prévia da transcrição:
=== Parte 1 ===
pode chamar. Acho que vai aguardar ali a transmissão. Só confirmar. Pode, agora tá ok. Senhoras e senhores,
muito boa tarde. Vamos dar início neste momento a nossa cerimônia. Pedimos por gentileza
para aqueles que ainda se encontram de pé, que procurem um lugar, que sente, que coloque
o seu celular em modo silencioso para o bom andamento desta cerimônia. Pedimos também
um pouco de si

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>